In [1]:
from spacepy import pycdf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import axes3d
import matplotlib
import matplotlib.colors as colors
from collections import Counter
from scipy.optimize import curve_fit
import uuid
import pandas
from requests import get  # to make GET request
import tarfile

In [2]:
cdf = pycdf.CDF('/Users/mayur/PhD/Cluster_data/C2_CP_FGM_SPIN__20080102_000000_20080504_000000_V140305.cdf')

In [3]:
for (i, item) in enumerate(cdf, start=0):
    print(i, item, cdf[i])

0 time_tags__C2_CP_FGM_SPIN CDF_EPOCH [2443077]
1 half_interval__C2_CP_FGM_SPIN CDF_FLOAT [2443077]
2 B_vec_xyz_gse__C2_CP_FGM_SPIN CDF_FLOAT [2443077, 3]
3 B_mag__C2_CP_FGM_SPIN CDF_FLOAT [2443077]
4 sc_pos_xyz_gse__C2_CP_FGM_SPIN CDF_FLOAT [2443077, 3]
5 range__C2_CP_FGM_SPIN CDF_INT4 [2443077]
6 tm__C2_CP_FGM_SPIN CDF_INT4 [2443077]
7 B_vec_xyz_gse__C2_CP_FGM_SPIN_REPRESENTATION_1 CDF_CHAR*1 [3] NRV
8 B_vec_xyz_gse__C2_CP_FGM_SPIN_LABEL_1 CDF_CHAR*2 [3] NRV
9 sc_pos_xyz_gse__C2_CP_FGM_SPIN_REPRESENTATION_1 CDF_CHAR*1 [3] NRV
10 sc_pos_xyz_gse__C2_CP_FGM_SPIN_LABEL_1 CDF_CHAR*1 [3] NRV


In [4]:
epoch = cdf[0][...]
vector = cdf[2][...]
position = cdf[4][...]

In [5]:
epsilon = 1.029
Bz = -0.35
Dp = 2.48
beta = 2.08
Mms = 6.96
a1 = 11.1266
a3 = -0.0005
a4 = 2.5966
a5 = 0.8182
a6 = -0.017
a7 = -0.0122
a8 = 1.3007
a9 = -0.0049
a10 = -0.0328
a11 = 6.047
a14 = -0.002
a12 = 1.029
Re = 6371
BS = 4*Re #to ensure we are not magnetically connected to the bow shock 

alpha = a5*(1+a6*Bz)*(1+a7*Dp)*(1+a10*np.log(1+beta))*(1+a14*Mms)
r0 = a1*(1+a3*Bz)*(1+a9*beta)*(1+a4*(((a8-1)*(Mms**2) + 2)/((a8+1)*(Mms**2))))*Dp**(-1/a11)
r0 = r0*Re
print(alpha, r0/Re)

0.7580145569010709 13.137494179562479


In [6]:
d = np.sqrt(position[:,0]**2 + position[:,1]**2 + position[:,2]**2)

theta = []
for i in range(len(epoch)):
    if position[i,0] > 0:
        theta_i = np.arcsin((np.sqrt(position[i,1]**2 + position[i,2]**2))/(np.sqrt(position[i,0]**2 + position[i,1]**2 + position[i,2]**2)))
    else:
        theta_i = np.pi - np.arcsin((np.sqrt(position[i,1]**2 + position[i,2]**2))/(np.sqrt(position[i,0]**2 + position[i,1]**2 + position[i,2]**2)))
    theta.append(theta_i)

In [7]:
r = r0*(((1+epsilon)/(1+epsilon*np.cos(theta))))**alpha
limit = d-(r+BS)

In [8]:
outside = []
outside_epoch = []

for i in range(len(epoch)):
    if limit[i] > 0:
        pristine = 1
        outside_epoch.append(i)
    else:
        pristine = 0
    outside.append(pristine)
    
print(Counter(outside))

start_times = []
end_times = []

for i in range(0,len(epoch)):
    if outside[i]>outside[i-1]:
        start_times.append(epoch[i])
    if outside[i]<outside[i-1]:
        end_times.append(epoch[i-1])

for (i, item) in enumerate(start_times, start=0):
    print(i, start_times[i], end_times[i])

Counter({0: 1574663, 1: 868414})
0 2008-01-02 20:48:13.577000 2008-01-03 01:18:01.525000
1 2008-01-05 03:34:33.436000 2008-01-05 12:16:06.339000
2 2008-01-07 11:11:05.509000 2008-01-07 22:26:03.104000
3 2008-01-09 19:07:05.702000 2008-01-10 08:20:31.772000
4 2008-01-12 03:14:26.150000 2008-01-12 18:08:50.546000
5 2008-01-14 11:29:19.453000 2008-01-15 03:49:07.770000
6 2008-01-16 19:49:58.070000 2008-01-17 13:19:48.293000
7 2008-01-19 04:16:05.491000 2008-01-19 22:45:20.885000
8 2008-01-21 12:47:33.482000 2008-01-22 08:09:08.236000
9 2008-01-23 21:22:59.412000 2008-01-24 17:31:51.720000
10 2008-01-26 06:01:12.868000 2008-01-27 02:52:41.133000
11 2008-01-28 23:30:14.955000 2008-01-29 12:10:51.258000
12 2008-01-30 23:23:25.990000 2008-01-31 21:26:06.523000
13 2008-02-02 08:07:16.393000 2008-02-03 06:38:46.964000
14 2008-02-04 22:00:21.607000 2008-02-05 15:50:20.558000
15 2008-02-07 01:41:44.816000 2008-02-08 01:03:26.092000
16 2008-02-09 10:37:46.142000 2008-02-09 15:04:39.103000
17 2008-

In [9]:
disconnected = []
connected = []

for i in range(len(epoch)):
    
    xm_max = position[i,0]+vector[i,0]*8*Re
    ym_max = position[i,1]+vector[i,1]*8*Re
    zm_max = position[i,2]+vector[i,2]*8*Re
    xm_min = position[i,0]-vector[i,0]*8*Re
    ym_min = position[i,1]-vector[i,1]*8*Re
    zm_min = position[i,2]-vector[i,2]*8*Re
    xm = np.linspace(xm_min,xm_max,2400)
    ym = np.linspace(ym_min,ym_max,2400)
    zm = np.linspace(zm_min,zm_max,2400)
    
    d = np.sqrt(xm**2+ym**2+zm**2)
    d_min = np.argmin(d)

    theta_t = np.arcsin((np.sqrt(ym[d_min]**2 + zm[d_min]**2))/(np.sqrt(xm[d_min]**2 + ym[d_min]**2 + zm[d_min]**2)))
    r_t = r0*(((1+epsilon)/(1+epsilon*np.cos(theta_t))))**alpha
    
    if np.min(d) < r_t or limit[i] < 0:
        dc = 0
    elif np.min(d) > r_t and limit[i] > 0:
        dc = 1
        
    disconnected.append(dc)
    
    if np.min(d) > r_t or limit[i] < 0:
        c = 0
    elif np.min(d) < r_t and limit[i] > 0:
        c = 1
    
    connected.append(c)
    
print(Counter(disconnected))
print(Counter(connected))

Counter({0: 1955606, 1: 487471})
Counter({0: 2062134, 1: 380943})


In [10]:
start_dc = []
end_dc = []
start_e = []
end_e = []

for i in range(0,len(epoch)):
    if disconnected[i]>disconnected[i-1] and disconnected[i]==disconnected[i+1]:
        start_e.append(i)
    if disconnected[i]<disconnected[i-1] and disconnected[i-1]==disconnected[i-2]:
        end_e.append(i-1)
        
for j in range(0,len(start_e)):
    if end_e[j]>(start_e[j]+60):
        start_dc.append(epoch[start_e[j]])
        end_dc.append(epoch[end_e[j]])

for (i, item) in enumerate(start_dc, start=0):
    print(i, start_dc[i], end_dc[i])

0 2008-01-02 20:48:13.577000 2008-01-02 20:58:21.400000
1 2008-01-02 21:20:49.361000 2008-01-02 21:25:09.857000
2 2008-01-02 21:30:40.645000 2008-01-02 21:36:32.107000
3 2008-01-02 21:40:40.198000 2008-01-02 22:04:26.722000
4 2008-01-02 22:05:08.071000 2008-01-02 22:42:25.026000
5 2008-01-02 23:59:52.600000 2008-01-03 01:09:57.747000
6 2008-01-03 01:10:47.365000 2008-01-03 01:18:01.525000
7 2008-01-05 03:34:33.436000 2008-01-05 03:43:35.107000
8 2008-01-05 03:51:22.350000 2008-01-05 03:57:46.895000
9 2008-01-05 04:00:03.347000 2008-01-05 04:12:40.032000
10 2008-01-05 04:13:33.786000 2008-01-05 05:23:05.895000
11 2008-01-05 05:23:18.300000 2008-01-05 05:44:19.443000
12 2008-01-05 05:44:35.982000 2008-01-05 05:57:54.017000
13 2008-01-05 06:01:00.088000 2008-01-05 06:23:52.873000
14 2008-01-05 06:26:34.134000 2008-01-05 06:50:49.618000
15 2008-01-05 06:51:02.023000 2008-01-05 07:01:42.932000
16 2008-01-05 07:01:55.336000 2008-01-05 07:06:11.700000
17 2008-01-05 07:06:19.970000 2008-01-05 

746 2008-03-02 02:53:06.878000 2008-03-02 03:01:39.896000
747 2008-03-02 04:06:24.760000 2008-03-02 04:13:10.210000
748 2008-03-02 04:13:22.621000 2008-03-02 04:27:14.206000
749 2008-03-02 04:31:10.029000 2008-03-02 04:42:53.359000
750 2008-03-02 04:44:57.476000 2008-03-02 04:50:11.906000
751 2008-03-02 04:50:20.180000 2008-03-02 05:01:22.138000
752 2008-03-02 05:01:42.825000 2008-03-02 05:07:38.627000
753 2008-03-02 05:07:46.901000 2008-03-02 05:19:46.781000
754 2008-03-02 05:22:28.133000 2008-03-02 05:33:34.229000
755 2008-03-02 05:34:03.189000 2008-03-02 06:07:46.299000
756 2008-03-02 06:08:15.260000 2008-03-02 06:18:23.434000
757 2008-03-02 06:24:48.198000 2008-03-02 06:36:56.352000
758 2008-03-02 06:37:17.038000 2008-03-02 06:42:52.154000
759 2008-03-02 06:43:12.841000 2008-03-02 06:59:54.053000
760 2008-03-02 07:00:18.876000 2008-03-02 07:09:29.129000
761 2008-03-02 07:12:55.991000 2008-03-02 07:35:57.830000
762 2008-03-02 07:36:14.379000 2008-03-02 08:09:32.666000
763 2008-03-02

1413 2008-04-30 18:24:16.646000 2008-04-30 18:33:02.298000
1414 2008-04-30 19:40:13.680000 2008-04-30 20:40:35.301000
1415 2008-05-02 23:30:50.494000 2008-05-02 23:50:42.537000
1416 2008-05-02 23:50:50.815000 2008-05-02 23:55:11.575000
1417 2008-05-03 00:30:47.319000 2008-05-03 00:35:28.773000
1418 2008-05-03 00:45:04.100000 2008-05-03 00:50:10.389000
1419 2008-05-03 00:50:18.667000 2008-05-03 01:03:04.389000
1420 2008-05-03 02:19:13.890000 2008-05-03 02:25:42.960000
1421 2008-05-03 03:07:02.246000 2008-05-03 03:11:47.839000
1422 2008-05-03 03:29:52.268000 2008-05-03 03:36:58.590000


In [11]:
start_c = []
end_c = []
start_ec = []
end_ec = []

for i in range(0,len(epoch)):
    if connected[i]>connected[i-1] and connected[i]==connected[i+1]:
        start_ec.append(i)
    if connected[i]<connected[i-1] and connected[i-1]==connected[i-2]:
        end_ec.append(i-1)
        
for j in range(0,len(start_ec)):
    if end_ec[j]>(start_ec[j]+60):
        start_c.append(epoch[start_ec[j]])
        end_c.append(epoch[end_ec[j]])

for (i, item) in enumerate(start_c, start=0):
    print(i, start_c[i], end_c[i])

0 2008-01-02 22:42:29.161000 2008-01-02 22:48:53.702000
1 2008-01-02 22:49:10.241000 2008-01-02 22:57:18.154000
2 2008-01-05 09:53:27.096000 2008-01-05 10:01:18.475000
3 2008-01-07 16:00:07.432000 2008-01-07 16:04:44.473000
4 2008-01-07 18:54:41.235000 2008-01-07 18:59:14.141000
5 2008-01-07 19:04:11.857000 2008-01-07 19:14:07.289000
6 2008-01-07 20:14:12.958000 2008-01-07 20:19:47.888000
7 2008-01-09 22:55:37.297000 2008-01-09 23:05:32.734000
8 2008-01-10 00:24:02.479000 2008-01-10 00:28:35.388000
9 2008-01-10 03:28:15.291000 2008-01-10 03:32:27.525000
10 2008-01-12 06:54:00.336000 2008-01-12 06:59:39.408000
11 2008-01-12 11:31:27.955000 2008-01-12 11:36:54.622000
12 2008-01-12 11:41:44.074000 2008-01-12 11:46:08.716000
13 2008-01-12 11:50:00.278000 2008-01-12 11:56:08.295000
14 2008-01-12 12:14:32.348000 2008-01-12 12:24:23.657000
15 2008-01-12 12:26:48.383000 2008-01-12 12:34:06.696000
16 2008-01-14 13:57:58.803000 2008-01-14 14:06:35.687000
17 2008-01-14 15:39:25.624000 2008-01-14 

434 2008-03-02 12:07:50.952000 2008-03-02 12:12:15.734000
435 2008-03-02 12:44:52.643000 2008-03-02 12:49:50.523000
436 2008-03-02 13:53:29.183000 2008-03-02 13:59:58.082000
437 2008-03-02 15:28:26.141000 2008-03-02 15:34:26.080000
438 2008-03-02 15:37:03.294000 2008-03-02 15:46:25.957000
439 2008-03-02 18:19:51.275000 2008-03-02 18:28:11.878000
440 2008-03-04 16:17:30.855000 2008-03-04 16:21:47.367000
441 2008-03-04 21:55:19.446000 2008-03-04 22:02:54.549000
442 2008-03-04 22:35:43.901000 2008-03-04 22:44:25.200000
443 2008-03-04 22:52:00.302000 2008-03-04 23:02:16.759000
444 2008-03-04 23:17:47.651000 2008-03-04 23:25:22.753000
445 2008-03-04 23:32:24.758000 2008-03-04 23:49:47.356000
446 2008-03-04 23:50:28.729000 2008-03-05 00:00:20.363000
447 2008-03-05 00:04:24.463000 2008-03-05 00:11:38.880000
448 2008-03-06 13:22:32.129000 2008-03-06 13:40:56.800000
449 2008-03-06 13:58:15.274000 2008-03-06 14:04:11.086000
450 2008-03-06 14:27:58.470000 2008-03-06 14:36:19.089000
451 2008-03-06

959 2008-04-11 07:20:34.909000 2008-04-11 07:25:32.851000
960 2008-04-11 07:25:41.128000 2008-04-11 07:30:59.760000
961 2008-04-11 07:36:18.392000 2008-04-11 07:42:18.406000
962 2008-04-11 07:48:14.281000 2008-04-11 08:04:26.731000
963 2008-04-11 08:13:03.991000 2008-04-11 08:21:45.390000
964 2008-04-11 08:23:12.290000 2008-04-11 08:27:49.541000
965 2008-04-11 09:02:47.550000 2008-04-11 09:18:51.723000
966 2008-04-11 09:18:59.999000 2008-04-11 10:03:20.787000
967 2008-04-11 11:55:16.895000 2008-04-11 12:04:31.398000
968 2008-04-11 12:16:23.148000 2008-04-11 12:21:41.781000
969 2008-04-11 12:42:48.034000 2008-04-11 12:54:48.060000
970 2008-04-11 13:17:50.180000 2008-04-11 13:22:02.603000
971 2008-04-11 13:26:39.854000 2008-04-11 13:31:37.796000
972 2008-04-11 14:05:25.456000 2008-04-11 14:11:13.055000
973 2008-04-11 14:15:54.445000 2008-04-11 14:21:17.215000
974 2008-04-11 14:21:25.491000 2008-04-11 14:30:19.304000
975 2008-04-11 14:45:42.096000 2008-04-11 14:52:02.800000
976 2008-04-11

In [12]:
date = pandas.Timestamp.date(start_dc[len(start_dc)-1])
year = date.year
filename = str(year)
np.savez(filename,start_dc,end_dc,start_c,end_c)